In [1]:
#import dependencies
import psycopg2
import pandas as pd
import datetime as dt

import warnings
warnings.filterwarnings('ignore')


In [2]:
# import resource files

co2_df = pd.read_csv('./Resources/global_co2_data_to_2019.csv')
temps_df = pd.read_csv('./Resources/global_temps_to_2016.csv')

In [3]:
co2_df

,iso_code,country,year,co2,co2_growth_prct,co2_growth_abs,consumption_co2,trade_co2,trade_co2_share,co2_per_capita,...,ghg_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,primary_energy_consumption,energy_per_capita,energy_per_gdp,population,gdp
0,AFG,Afghanistan,1949,0.015,NaN,NaN,NaN,NaN,NaN,0.002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7663783.0,NaN
1,AFG,Afghanistan,1950,0.084,475.000,0.070,NaN,NaN,NaN,0.011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7752000.0,1.949480e+10
2,AFG,Afghanistan,1951,0.092,8.696,0.007,NaN,NaN,NaN,0.012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7840000.0,2.006385e+10
3,AFG,Afghanistan,1952,0.092,NaN,NaN,NaN,NaN,NaN,0.012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7936000.0,2.074235e+10
4,AFG,Afghanistan,1953,0.106,16.000,0.015,NaN,NaN,NaN,0.013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8040000.0,2.201546e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21011,ZWE,Zimbabwe,2015,12.170,1.653,0.198,13.308,1.138,9.350,0.881,...,4.885,11.87,0.859,6.68,0.484,NaN,NaN,NaN,13815000.0,2.503057e+10
21012,ZWE,Zimbabwe,2016,10.815,-11.139,-1.356,12.171,1.356,12.542,0.771,...,4.703,11.92,0.850,6.55,0.467,NaN,NaN,NaN,14030000.0,2.515176e+10
21013,ZWE,Zimbabwe,2017,10.247,-5.251,-0.568,11.774,1.527,14.902,0.720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14237000.0,NaN
21014,ZWE,Zimbabwe,2018,11.341,10.674,1.094,12.815,1.475,13.006,0.785,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14439000.0,NaN


In [4]:
temps_df

,iso_code,country,year,temperature(C)
0,AFG,Afghanistan,1901,13.172872
1,AFG,Afghanistan,1902,13.342649
2,AFG,Afghanistan,1903,12.006417
3,AFG,Afghanistan,1904,12.531729
4,AFG,Afghanistan,1905,12.267143
...,...,...,...,...
22731,ZWE,Zimbabwe,2012,21.910075
22732,ZWE,Zimbabwe,2013,21.624350
22733,ZWE,Zimbabwe,2014,21.710483
22734,ZWE,Zimbabwe,2015,22.327625


In [5]:
# find which columns in co2_df are the least complete, aka have the most NAs

percent_missing = co2_df.isnull().sum()* 100/len(co2_df)
percent_missing.sort_values()

country                                 0.000000
year                                    0.000000
cumulative_co2                          0.299772
share_global_cumulative_co2             0.299772
iso_code                                1.551199
population                              1.912828
co2                                     2.345832
share_global_co2                        2.345832
co2_per_capita                          4.173011
co2_growth_abs                          7.085078
co2_growth_prct                         7.522840
share_global_cumulative_oil_co2        13.908451
cumulative_oil_co2                     13.908451
share_global_oil_co2                   15.107537
oil_co2                                15.107537
oil_co2_per_capita                     16.401789
cumulative_coal_co2                    24.195851
share_global_cumulative_coal_co2       24.195851
coal_co2                               31.409402
share_global_coal_co2                  31.409402
coal_co2_per_capita 

In [6]:
# drop all columns with over 40% missing values

co2_df = co2_df.loc[:, co2_df.isnull().sum()* 100/len(co2_df) < 40]
co2_df        

,iso_code,country,year,co2,co2_growth_prct,co2_growth_abs,co2_per_capita,share_global_co2,cumulative_co2,share_global_cumulative_co2,...,coal_co2_per_capita,oil_co2_per_capita,share_global_coal_co2,share_global_oil_co2,cumulative_coal_co2,cumulative_oil_co2,share_global_cumulative_coal_co2,share_global_cumulative_oil_co2,population,gdp
0,AFG,Afghanistan,1949,0.015,NaN,NaN,0.002,0.000,0.015,0.000,...,0.002,NaN,0.000,NaN,0.015,NaN,0.000,NaN,7663783.0,NaN
1,AFG,Afghanistan,1950,0.084,475.000,0.070,0.011,0.001,0.099,0.000,...,0.003,0.008,0.001,0.004,0.036,0.063,0.000,0.000,7752000.0,1.949480e+10
2,AFG,Afghanistan,1951,0.092,8.696,0.007,0.012,0.001,0.191,0.000,...,0.003,0.008,0.001,0.004,0.061,0.129,0.000,0.000,7840000.0,2.006385e+10
3,AFG,Afghanistan,1952,0.092,NaN,NaN,0.012,0.001,0.282,0.000,...,0.004,0.008,0.001,0.003,0.093,0.189,0.000,0.001,7936000.0,2.074235e+10
4,AFG,Afghanistan,1953,0.106,16.000,0.015,0.013,0.002,0.388,0.000,...,0.005,0.008,0.001,0.003,0.131,0.257,0.000,0.001,8040000.0,2.201546e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21011,ZWE,Zimbabwe,2015,12.170,1.653,0.198,0.881,0.035,726.480,0.048,...,0.582,0.263,0.055,0.030,577.422,131.580,0.081,0.025,13815000.0,2.503057e+10
21012,ZWE,Zimbabwe,2016,10.815,-11.139,-1.356,0.771,0.031,737.295,0.048,...,0.503,0.226,0.049,0.027,584.482,134.753,0.080,0.025,14030000.0,2.515176e+10
21013,ZWE,Zimbabwe,2017,10.247,-5.251,-0.568,0.720,0.029,747.542,0.047,...,0.448,0.231,0.044,0.027,590.865,138.036,0.080,0.025,14237000.0,NaN
21014,ZWE,Zimbabwe,2018,11.341,10.674,1.094,0.785,0.031,758.882,0.047,...,0.510,0.235,0.050,0.028,598.234,141.426,0.079,0.025,14439000.0,NaN


In [7]:
percent_missing = co2_df.isnull().sum()* 100/len(co2_df)
percent_missing.sort_values()

country                              0.000000
year                                 0.000000
cumulative_co2                       0.299772
share_global_cumulative_co2          0.299772
iso_code                             1.551199
population                           1.912828
share_global_co2                     2.345832
co2                                  2.345832
co2_per_capita                       4.173011
co2_growth_abs                       7.085078
co2_growth_prct                      7.522840
share_global_cumulative_oil_co2     13.908451
cumulative_oil_co2                  13.908451
share_global_oil_co2                15.107537
oil_co2                             15.107537
oil_co2_per_capita                  16.401789
share_global_cumulative_coal_co2    24.195851
cumulative_coal_co2                 24.195851
coal_co2                            31.409402
share_global_coal_co2               31.409402
coal_co2_per_capita                 32.289684
co2_per_gdp                       

In [15]:
# keep only most relevant columns

clean_co2 = co2_df.drop(['iso_code','share_global_cumulative_oil_co2', 'cumulative_oil_co2', 'share_global_oil_co2','oil_co2', 'oil_co2_per_capita','share_global_cumulative_coal_co2','cumulative_coal_co2','coal_co2','share_global_coal_co2','coal_co2_per_capita'], axis=1)
clean_co2

,country,year,co2,co2_growth_prct,co2_growth_abs,co2_per_capita,share_global_co2,cumulative_co2,share_global_cumulative_co2,co2_per_gdp,population,gdp
0,Afghanistan,1949,0.015,NaN,NaN,0.002,0.000,0.015,0.000,NaN,7663783.0,NaN
1,Afghanistan,1950,0.084,475.000,0.070,0.011,0.001,0.099,0.000,0.004,7752000.0,1.949480e+10
2,Afghanistan,1951,0.092,8.696,0.007,0.012,0.001,0.191,0.000,0.005,7840000.0,2.006385e+10
3,Afghanistan,1952,0.092,NaN,NaN,0.012,0.001,0.282,0.000,0.004,7936000.0,2.074235e+10
4,Afghanistan,1953,0.106,16.000,0.015,0.013,0.002,0.388,0.000,0.005,8040000.0,2.201546e+10
...,...,...,...,...,...,...,...,...,...,...,...,...
21011,Zimbabwe,2015,12.170,1.653,0.198,0.881,0.035,726.480,0.048,0.486,13815000.0,2.503057e+10
21012,Zimbabwe,2016,10.815,-11.139,-1.356,0.771,0.031,737.295,0.048,0.430,14030000.0,2.515176e+10
21013,Zimbabwe,2017,10.247,-5.251,-0.568,0.720,0.029,747.542,0.047,NaN,14237000.0,NaN
21014,Zimbabwe,2018,11.341,10.674,1.094,0.785,0.031,758.882,0.047,NaN,14439000.0,NaN


In [16]:
clean_co2 = clean_co2.dropna()
clean_co2

,country,year,co2,co2_growth_prct,co2_growth_abs,co2_per_capita,share_global_co2,cumulative_co2,share_global_cumulative_co2,co2_per_gdp,population,gdp
1,Afghanistan,1950,0.084,475.000,0.070,0.011,0.001,0.099,0.000,0.004,7752000.0,1.949480e+10
2,Afghanistan,1951,0.092,8.696,0.007,0.012,0.001,0.191,0.000,0.005,7840000.0,2.006385e+10
4,Afghanistan,1953,0.106,16.000,0.015,0.013,0.002,0.388,0.000,0.005,8040000.0,2.201546e+10
6,Afghanistan,1955,0.154,44.828,0.048,0.019,0.002,0.649,0.000,0.007,8271000.0,2.292989e+10
7,Afghanistan,1956,0.183,19.048,0.029,0.022,0.002,0.832,0.000,0.008,8399000.0,2.395993e+10
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,Zimbabwe,2012,7.659,-19.213,-1.821,0.584,0.022,690.720,0.049,0.374,13115000.0,2.048226e+10
21009,Zimbabwe,2013,11.617,51.675,3.958,0.870,0.033,702.337,0.049,0.489,13350000.0,2.374258e+10
21010,Zimbabwe,2014,11.973,3.065,0.356,0.881,0.034,714.309,0.048,0.484,13587000.0,2.474828e+10
21011,Zimbabwe,2015,12.170,1.653,0.198,0.881,0.035,726.480,0.048,0.486,13815000.0,2.503057e+10


In [17]:
# merge temp and co2 dataframes

co2_temp = pd.merge(clean_co2, temps_df, how= 'left', on = ['country', 'year'])
co2_temp

,country,year,co2,co2_growth_prct,co2_growth_abs,co2_per_capita,share_global_co2,cumulative_co2,share_global_cumulative_co2,co2_per_gdp,population,gdp,iso_code,temperature(C)
0,Afghanistan,1950,0.084,475.000,0.070,0.011,0.001,0.099,0.000,0.004,7752000.0,1.949480e+10,AFG,11.619864
1,Afghanistan,1951,0.092,8.696,0.007,0.012,0.001,0.191,0.000,0.005,7840000.0,2.006385e+10,AFG,12.647118
2,Afghanistan,1953,0.106,16.000,0.015,0.013,0.002,0.388,0.000,0.005,8040000.0,2.201546e+10,AFG,13.272074
3,Afghanistan,1955,0.154,44.828,0.048,0.019,0.002,0.649,0.000,0.007,8271000.0,2.292989e+10,AFG,13.275183
4,Afghanistan,1956,0.183,19.048,0.029,0.022,0.002,0.832,0.000,0.008,8399000.0,2.395993e+10,AFG,12.785808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12478,Zimbabwe,2012,7.659,-19.213,-1.821,0.584,0.022,690.720,0.049,0.374,13115000.0,2.048226e+10,ZWE,21.910075
12479,Zimbabwe,2013,11.617,51.675,3.958,0.870,0.033,702.337,0.049,0.489,13350000.0,2.374258e+10,ZWE,21.624350
12480,Zimbabwe,2014,11.973,3.065,0.356,0.881,0.034,714.309,0.048,0.484,13587000.0,2.474828e+10,ZWE,21.710483
12481,Zimbabwe,2015,12.170,1.653,0.198,0.881,0.035,726.480,0.048,0.486,13815000.0,2.503057e+10,ZWE,22.327625


In [18]:
# reorder columns for my sanity
co2_temp = co2_temp[['iso_code', 'country', 'year', 'temperature(C)', 'population', 'gdp', 'co2', 'share_global_co2', 'co2_per_capita', 'co2_per_gdp', 'co2_growth_prct','co2_growth_abs', 'cumulative_co2', 'share_global_cumulative_co2' ]]
co2_temp

,iso_code,country,year,temperature(C),population,gdp,co2,share_global_co2,co2_per_capita,co2_per_gdp,co2_growth_prct,co2_growth_abs,cumulative_co2,share_global_cumulative_co2
0,AFG,Afghanistan,1950,11.619864,7752000.0,1.949480e+10,0.084,0.001,0.011,0.004,475.000,0.070,0.099,0.000
1,AFG,Afghanistan,1951,12.647118,7840000.0,2.006385e+10,0.092,0.001,0.012,0.005,8.696,0.007,0.191,0.000
2,AFG,Afghanistan,1953,13.272074,8040000.0,2.201546e+10,0.106,0.002,0.013,0.005,16.000,0.015,0.388,0.000
3,AFG,Afghanistan,1955,13.275183,8271000.0,2.292989e+10,0.154,0.002,0.019,0.007,44.828,0.048,0.649,0.000
4,AFG,Afghanistan,1956,12.785808,8399000.0,2.395993e+10,0.183,0.002,0.022,0.008,19.048,0.029,0.832,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12478,ZWE,Zimbabwe,2012,21.910075,13115000.0,2.048226e+10,7.659,0.022,0.584,0.374,-19.213,-1.821,690.720,0.049
12479,ZWE,Zimbabwe,2013,21.624350,13350000.0,2.374258e+10,11.617,0.033,0.870,0.489,51.675,3.958,702.337,0.049
12480,ZWE,Zimbabwe,2014,21.710483,13587000.0,2.474828e+10,11.973,0.034,0.881,0.484,3.065,0.356,714.309,0.048
12481,ZWE,Zimbabwe,2015,22.327625,13815000.0,2.503057e+10,12.170,0.035,0.881,0.486,1.653,0.198,726.480,0.048


In [19]:
# clean nas from combined dataset

co2_temp = co2_temp.dropna()
co2_temp

,iso_code,country,year,temperature(C),population,gdp,co2,share_global_co2,co2_per_capita,co2_per_gdp,co2_growth_prct,co2_growth_abs,cumulative_co2,share_global_cumulative_co2
0,AFG,Afghanistan,1950,11.619864,7752000.0,1.949480e+10,0.084,0.001,0.011,0.004,475.000,0.070,0.099,0.000
1,AFG,Afghanistan,1951,12.647118,7840000.0,2.006385e+10,0.092,0.001,0.012,0.005,8.696,0.007,0.191,0.000
2,AFG,Afghanistan,1953,13.272074,8040000.0,2.201546e+10,0.106,0.002,0.013,0.005,16.000,0.015,0.388,0.000
3,AFG,Afghanistan,1955,13.275183,8271000.0,2.292989e+10,0.154,0.002,0.019,0.007,44.828,0.048,0.649,0.000
4,AFG,Afghanistan,1956,12.785808,8399000.0,2.395993e+10,0.183,0.002,0.022,0.008,19.048,0.029,0.832,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12478,ZWE,Zimbabwe,2012,21.910075,13115000.0,2.048226e+10,7.659,0.022,0.584,0.374,-19.213,-1.821,690.720,0.049
12479,ZWE,Zimbabwe,2013,21.624350,13350000.0,2.374258e+10,11.617,0.033,0.870,0.489,51.675,3.958,702.337,0.049
12480,ZWE,Zimbabwe,2014,21.710483,13587000.0,2.474828e+10,11.973,0.034,0.881,0.484,3.065,0.356,714.309,0.048
12481,ZWE,Zimbabwe,2015,22.327625,13815000.0,2.503057e+10,12.170,0.035,0.881,0.486,1.653,0.198,726.480,0.048
